In [2]:
sc.master

u'local[*]'

In [3]:
global Path    
if sc.master[0:5]=="local" :
   Path="file:/home/hduser/pythonsparkexample/PythonProject/"
else:   
   Path="hdfs://master:9000/user/hduser/"

In [4]:
print("開始匯入資料...")
rawDataWithHeader = sc.textFile(Path+"data/DRtrain.csv")
header = rawDataWithHeader.first() 
rawData = rawDataWithHeader.filter(lambda x:x !=header)    
lines = rawData.map(lambda x: x.split(","))
print("共計：" + str(lines.count()) + "筆")

開始匯入資料...
共計：42000筆


In [5]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np

def extract_label(field):
    label=(field[0])
    return float(label)

def extract_features(x):
    categoryFeatures = [float(a) for a in x[1:]]
    return categoryFeatures
    
def PrepareData(sc): 
    #----------------------1.匯入並轉換資料-------------
    global Path    
    if sc.master[0:5]=="local" :
       Path="file:/home/hduser/pythonsparkexample/PythonProject/"
    else:   
       Path="hdfs://master:9000/user/hduser/"

    print("開始匯入資料...")
    rawDataWithHeader = sc.textFile(Path+"data/DRtrain.csv")
    header = rawDataWithHeader.first() 
    rawData = rawDataWithHeader.filter(lambda x:x !=header)    
    lines = rawData.map(lambda x: x.split(","))
    print("共計：" + str(lines.count()) + "筆")
     #----------2.建立訓練評估所需資料 RDD[LabeledPoint]-------    
    labelpointRDD = lines.map(lambda r: LabeledPoint(extract_label(r), extract_features(r)))
        
    #-----------3.以隨機方式將資料分為3部份並且回傳-------------
    (trainData, validationData, testData) = labelpointRDD.randomSplit([8, 1, 1])
    print("將資料分為trainData:" + str(trainData.count()) + 
              "   validationData:" + str(validationData.count()) +
              "   testData:" + str(testData.count()))
    return (trainData, validationData, testData) #回傳資料

(trainData, validationData, testData) =PrepareData(sc)
trainData.persist()
validationData.persist()
testData.persist()

開始匯入資料...
共計：42000筆
將資料分為trainData:33425   validationData:4310   testData:4265


PythonRDD[13] at RDD at PythonRDD.scala:48

In [6]:
from pyspark.mllib.tree import DecisionTree
import matplotlib.pyplot as plt
import numpy as np

#images=dataRDD, idx=0, num=顯示筆數
def plot_image(images, idx, num):
    image_data = images.take(num)
    fig = plt.gcf()
    fig.set_size_inches(12,14)
    if num>25:
        num = 25
    for i in range(0,num):
        target = image_data[idx]
        prediction = model.predict(target[1])
        ax = plt.subplot(5,5,1+i)
        ax.imshow(target[0], cmap='binary')
        title = "prediction:"+ str(prediction)
        ax.set_title(title, fontsize=10)
        ax.set_xticks([]);ax.set_yticks([])
        idx += 1
    plt.show()

model=DecisionTree.trainClassifier( \
        trainData, numClasses=10, categoricalFeaturesInfo={}, \
        impurity="entropy", maxDepth=5, maxBins=5)

def extract_features1(x):
    categoryFeatures = [float(a) for a in x[:]]
    return categoryFeatures

def PredictData(sc,model): 
    print("開始匯入資料...")
    rawDataWithHeader = sc.textFile(Path+"data/DRtest.csv")
    header = rawDataWithHeader.first() 
    rawData = rawDataWithHeader.filter(lambda x:x !=header)
    line = rawData.map(lambda x: x.split(","))
    print("共計：" + str(line.count()) + "筆")
    dataRDD = line.map(lambda r:(np.array(extract_features1(r)).reshape((28,28)), extract_features1(r)))
    DescDict = {
        0: "0",
        1: "1",
        2: "2",
        3: "3",
        4: "4",
        5: "5",
        6: "6",
        7: "7",
        8: "8",
        9: "9" 
     }
    plot_image(dataRDD, 0, 10)

print("==========預測資料===============")
PredictData(sc, model)

==========預測資料===============
開始匯入資料...
共計：28000筆
